# Setup

In [1]:
# Basic Python

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from tqdm import tqdm

# Quant Libraries

import yfinance as yf # yahoo finance PyPi for getting stock data

In [150]:
def curr_val(df):

    time = datetime.now()

    date = df['real_date']
    value = df['value']

    index = 0

    while df['active'][index] == 1:
        index += 1

    week_prtf = curr_prtf(df, date[index - 1])

    old_price = prtf_price(week_prtf, date[index - 1], True)
    new_price = prtf_price(week_prtf, datetime.now(), False)

    val = value[index - 1] * new_price / old_price

    return (val - 1) * 100, time

def curr_prtf(df, date):

    stocks = stock_list(df)
    prtf = {}

    for i in range(len(stocks)):

        prtf[stocks[i]] = df.loc[df['real_date'] == date, stocks[i]].values[0]

    return prtf

def prtf_price(prtf, date, morning):

    tot_price = 0
    stocks = list(prtf.keys())
    values = list(prtf.values())

    for i in range(len(prtf)):

        tot_price += price(stocks[i], date, morning) * values[i]

    return tot_price

def price(stk, date, morning):

    if morning:
        return yf.download(stk, start=date, end=date + timedelta(1), interval='1d', progress=False)['Open'].values[0]
    else:
        return yf.download(stk, start=date, end=date, interval='1d' , progress=False)['Close'].values[0]

def update_row(df):

    index = 0

    while df['active'][index] == 1:
        index += 1
    if df['real_date'][index] > datetime.now():
        print("It is not recent enough to make that change")

    else:
        print("Making that change")

        stocks = stock_list(df)

        for j in range(len(stocks)):

            new_price = price(stocks[j], df['real_date'][index + 1], True)
            old_price = price(stocks[j], df['real_date'][index], True)

            df[stocks[j]][index - 1] = df[stocks[j]][index - 1] * new_price / old_price

        df['value'][index] = df['value'][index] * curr_prtf(df, df['real_date'][index]) / curr_prtf(df, df['real_date'][index - 1])

        sum = 0

        for k in range(len(stocks)):

            sum += df[stocks[j]][index - 1]

        for q in range(len(stocks)):

            df[stocks[j]][index] = df[stocks[j]][index - 1] / sum

        print("Change made")

def stock_list(df):

    lst = df.columns.to_list()
    lst.remove('real_date')
    return lst[3:]

prtf_df = pd.read_csv('equate_prtf.csv')
date_format = "%Y-%m-%d"
prtf_df['real_date'] = None
for i in range(len(prtf_df)):
    prtf_df['real_date'][i] = datetime.strptime(prtf_df['date'][i], date_format)

<ipython-input-150-e806d446d686>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prtf_df['real_date'][i] = datetime.strptime(prtf_df['date'][i], date_format)
<ipython-input-150-e806d446d686>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prtf_df['real_date'][i] = datetime.strptime(prtf_df['date'][i], date_format)
<ipython-input-150-e806d446d686>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prtf_df['real_date'][i] = datetime.s

# Current Performance

In [120]:
print(stock_list(prtf_df))

['L.TO', 'DOL.TO', 'KILO.TO', 'VFV.TO']


In [154]:
results, time = curr_val(prtf_df)

print("The EQUATE Fund has returned ", round(curr_val(prtf_df)[0], 3), "% at time ", time)

The EQUATE Fund has returned  1.756 % at time  2024-01-19 15:32:51.871162


## S&P500 Benchmark

In [155]:
today = datetime.now()
date_format = "%Y-%m-%d"
index_fund = 'VFV.TO'
start = datetime.strptime('2024-01-15', date_format)
og_price = yf.download(index_fund, start=start, end=start + timedelta(1), interval='1d', progress=False)['Open'].values[0]
curr_price = yf.download(index_fund, start=today, end=today + timedelta(1), interval='1d', progress=False)['Close'].values[0]

print("The S&P500 has returned ", round(100 * (curr_price - og_price) / og_price, 3), "%")

The S&P500 has returned  1.23 %


# Testing